In [1]:
from src.acoustic_analyser import frame
import numpy as np

test_frame = frame.from_file(
    member_file="test_member.json", constraint_file="test_constraint.csv", debug=False
)

free_end = test_frame.fixed_end(member_id=0)
free_end = test_frame.free_end(member_id=1)

In [2]:
test_frame.get_determinant(w=10)

(-0.22419369236865638-0.008377076802613428j)


(-0.22419369236865638-0.008377076802613428j)

In [ ]:
# _ = test_frame.get_frequency_graph(lower_limit=1, upper_limit=100, step_size=2)

In [3]:
natural_freq = (
    lambda x: np.abs(test_frame.get_determinant(w=np.abs(x) * 2 * np.pi)) ** 2
)

In [4]:
natural_freq(-1)

(-0.20643703964498247-0.04028903132113413j)


0.04423945738217938

In [ ]:
test_frame.get_natural_frequency(lower_limit=10, upper_limit=12)

In [ ]:
# test_frame.get_natural_frequency(lower_limit=50, upper_limit=55)

In [ ]:
_ = test_frame.get_params_solution(natural_freq=11.16)

In [ ]:
from src.modules.bc import fixed_end

step_size = 0.2
constraint: fixed_end = test_frame.constraints[1]
member_curr = constraint.member
points = np.append(
    np.arange(step_size, member_curr.length, step_size), member_curr.length
)